# Data

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df_adult = pd.read_csv('adult.csv')

In [ ]:
df_adult.replace('?',np.nan,inplace = True)

PREPROCESS

In [ ]:
adult_train = df_adult[:20000]
adult_test = df_adult[20000:]
adult_test.drop(columns = ['income'], inplace = True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
adult_train.to_csv('adult_train.csv', index = False)
adult_test.to_csv('adult_test.csv', index = False)

# Additional Liblary

In [ ]:
!pip install category_encoders

     |████████████████████████████████| 81kB 4.7MB/s 


# Model Training

In [ ]:
# Basic Operations
import pandas as pd
import numpy as np

# ML Models
from sklearn.linear_model import LogisticRegression

# Feature Engineering
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import category_encoders as ce

# Evaluation
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Model
import pickle
import joblib

# data
adult_train = pd.read_csv('adult_train.csv')

# preprocess
binary_encoder_pipeline = Pipeline([
                                    ('imputer',SimpleImputer(strategy = 'constant',fill_value = 'NC')),
                                    ('binary encoder',ce.BinaryEncoder())
])

transformer = ColumnTransformer([
                                 ('one hot encoder',OneHotEncoder(drop = 'first'),['relationship','race','sex']),
                                 ('binary encoder',binary_encoder_pipeline,['workclass','marital.status','occupation','native.country'])
                                ], remainder = 'passthrough')

# Data Splitting
X = df_adult.drop(columns = ['fnlwgt','income','education'])
y = np.where(df_adult['income']=='>50K',1,0)

# Model Selection
model = LogisticRegression(solver = 'liblinear', random_state = 2020)

estimator = Pipeline([
                      ('preprocess',transformer),
                      ('clf',model)
])

hyperparam_space = {
    'clf__C':[100, 10, 1, 0.1, 0.01, 0.001],
    'clf__solver':['liblinear','newton-cg']
}

skfold = StratifiedKFold(n_splits = 5)

grid_search = GridSearchCV(
    estimator, # model to tune
    param_grid = hyperparam_space, # hyperparameter space
    cv = skfold, # evaluation method
    scoring = 'f1', # metrics
    n_jobs = -1 # use all cores
)
grid_search.fit(X, y)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/lib/python3.6/dist-packages/scipy/optimize/linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.6/dist-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.6/dist-packages/scipy/optimize/linesearch.py:466: LineSearchWarning: The line search algorithm did not converge

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocess',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='passthrough',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('one '
                                                                         'hot '
                                                                         'encoder',
                                                                         OneHotEncoder(categories='auto',
                                                                                       drop='first',
                       

# Prediction : Pickle

In [ ]:
# Model Pickling
grid_search.best_estimator_.fit(X,y) # FINAL MODEL
filename = 'Model Final.sav'
pickle.dump(grid_search.best_estimator_,open(filename,'wb'))

# load dataset
df_adult_testing = pd.read_csv('adult_test.csv')
df_adult_testing= df_adult_testing.drop(columns = ['fnlwgt','education'])

# load model
filename = 'Model Final.sav'
loaded_model = pickle.load(open(filename,'rb'))

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/lib/python3.6/dist-packages/scipy/optimize/linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.6/dist-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.6/dist-packages/scipy/optimize/linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.6/dist-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The l

> ## Predict Test Dataset

In [ ]:
loaded_model.predict(df_adult_testing)

array([0, 1, 0, ..., 0, 0, 0])

In [ ]:
loaded_model.predict(df_adult_testing[0:1])

array([0])

> ## Predict an Observation

In [ ]:
df_predict = pd.DataFrame({
    'age':[43],
    'workclass':['Self-emp-not-inc'],
    'education.num':[13],
    'marital.status':['Separated'],
    'occupation':['Craft-repair'],
    'relationship':['Unmarried'],
    'race':['White'],
    'sex':['Male'],
    'capital.gain':[0],
    'capital.loss':[0],
    'hours.per.week':[35],
    'native.country':['United-States']
})

print('predict class :',loaded_model.predict(df_predict))
print('predict proba :',loaded_model.predict_proba(df_predict))

predict class : [0]
predict proba : [[0.87146373 0.12853627]]


# Prediction Joblib

In [ ]:
# saving model
joblib.dump(grid_search.best_estimator_, "model joblib")

# loading model
model_joblib = joblib.load("model joblib")

> ## Predict an Observation

In [ ]:
df_predict = pd.DataFrame({
    'age':[43],
    'workclass':['Self-emp-not-inc'],
    'education.num':[13],
    'marital.status':['Separated'],
    'occupation':['Craft-repair'],
    'relationship':['Unmarried'],
    'race':['White'],
    'sex':['Male'],
    'capital.gain':[0],
    'capital.loss':[0],
    'hours.per.week':[35],
    'native.country':['United-States']
})

print('predict class :',loaded_model.predict(df_predict))
print('predict proba :',loaded_model.predict_proba(df_predict))

predict class : [0]
predict proba : [[0.87146373 0.12853627]]
